In [4]:
import requests
import pandas as pd
import numpy as np
import json


## **Seccion 1:** Descargue de datos
*****
Se descargaran los datos directamente de la  pagina [datos.gov](https://datos.gov.co/api/views/eh75-8ah6/rows.csv?accessType=DOWNLOAD), donde se almacenaran bajo el nombre de `datos_sinLimpiar.csv`

**Nota:** Esto puede tomar un tiempo

In [ ]:
url = "https://datos.gov.co/api/views/eh75-8ah6/rows.csv?accessType=DOWNLOAD"

#Extraccion de los datos dado url
response = requests.get(url)

if response.status_code == 200:
  with open('Content/datos_sinLimpiar.csv','wb') as archivo:
    archivo.write(response.content)


#### Comprobacion de los datos descargados
****




In [5]:
datos = pd.read_csv('Content/datos_sinLimpiar.csv')
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,FECHA_DESPACHO,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,11001,15759,03/01/2021 12:00:00 AM,8,TRANSITO,2,0
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,11001,76001,03/12/2021 12:00:00 AM,11,ORIGEN,4,45
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,01/09/2021 12:00:00 AM,6,ORIGEN,1,3
3,T.T. DE MELGAR,BUS,BASICO,73001,11001,08/13/2021 12:00:00 AM,16,TRANSITO,1,10
4,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,02/28/2021 12:00:00 AM,12,ORIGEN,1,1


### Datos de muestreo
****
Dado a al cantidad masiva de datos se usara una muestra aleatoria de estos a la cual se le aplicara la limpieza de los datos

In [6]:
print(f"Longitud de datos originales {len(datos)}")
#para este caso tomaremos el 50% de los datos totales
datos = datos[:int(len(datos)*0.5)]
print(f"longitud de la muestra {len(datos)}")

Longitud de datos originales 22169927
longitud de la muestra 11084963


### Subseccion 1.1: Descargue de datos de diviPola
****
Se extraeran los datos del codigo municipal para su correcta traduccion en los `datos_sinLimpiar.csv`,donde a su vez esto se almacenaran en un arhivo ..csv llamado `diccionario_municipios.csv`

In [6]:
url = "https://datos.gov.co/api/views/gdxc-w37w/rows.csv?accessType=DOWNLOAD"

#Extraccion de los datos dado url
response = requests.get(url)

if response.status_code == 200:
  with open('Content/diccionario_municipios.csv','wb') as archivo:
    archivo.write(response.content)

#### Comprobacion de los datos descargados
****


In [7]:
dicc = pd.read_csv("Content/diccionario_municipios.csv")
dicc.head()

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001.0,ANTIOQUIA,MEDELLÍN,Municipio
1,05,5002.0,ANTIOQUIA,ABEJORRAL,Municipio
2,05,5004.0,ANTIOQUIA,ABRIAQUÍ,Municipio
3,05,5021.0,ANTIOQUIA,ALEJANDRÍA,Municipio
4,05,5030.0,ANTIOQUIA,AMAGÁ,Municipio


## **Seccion 2: Limpieza de datos y creacion del csv para humanos**
***
En esta seccion se vera lo siguiente:

1. Limpieza de datos nulos
1. Cambio de numero a datos de `MUNICIPIO_ORIGEN_RUTA` y	`MUNICIPIO_DESTINO_RUTA`
1. Separacion de la columna *fecha_despacho* por *mes_despacho*, *dia_despacho* y *ano_despacho*
1. Encoding de datos *Categoricos*
1. Creacion del archivo `diccionario_municipios.json` para los datos de `diccionario_municipio.csv`
1. Descargue del nuevo dataset con nombre de `datos_limpios.csv`

In [8]:
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,FECHA_DESPACHO,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,11001,15759,03/01/2021 12:00:00 AM,8,TRANSITO,2,0
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,11001,76001,03/12/2021 12:00:00 AM,11,ORIGEN,4,45
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,01/09/2021 12:00:00 AM,6,ORIGEN,1,3
3,T.T. DE MELGAR,BUS,BASICO,73001,11001,08/13/2021 12:00:00 AM,16,TRANSITO,1,10
4,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,02/28/2021 12:00:00 AM,12,ORIGEN,1,1


**1. Limpieza de datos nulos**
***


 * **deteccion de los datos**

In [9]:
datos.isnull().sum()

TERMINAL                  0
CLASE_VEHICULO            0
NIVEL_SERVICIO            0
MUNICIPIO_ORIGEN_RUTA     0
MUNICIPIO_DESTINO_RUTA    0
FECHA_DESPACHO            0
HORA_DESPACHO             0
TIPO_DESPACHO             0
DESPACHOS                 0
PASAJEROS                 0
dtype: int64

**2.5 Traducir del codigo de municipio a su respectivo nombre**
***

In [11]:
diccionario=np.array(dicc.iloc[:,[1,3]])
diccionario = dict(diccionario)

In [12]:
datos['MUNICIPIO_ORIGEN_RUTA'] = [ diccionario[dato] for dato in datos['MUNICIPIO_ORIGEN_RUTA']]
datos['MUNICIPIO_DESTINO_RUTA'] = [ diccionario[dato] for dato in datos['MUNICIPIO_DESTINO_RUTA']]

**3. Separacion de la columna *fecha_despacho* por *mes_despacho*, *dia_despacho* y *hora_despacho***
***

In [13]:
datos[['fecha','hora','tipo_hora']]=datos['FECHA_DESPACHO'].str.split(' ',expand=True)
datos.drop(columns=['FECHA_DESPACHO'],inplace=True)

In [14]:
datos[['dia_despacho','mes_despacho','ano_despacho']] = datos['fecha'].str.split('/',expand=True)
datos.drop(columns=['fecha'],inplace=True)
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,hora,tipo_hora,dia_despacho,mes_despacho,ano_despacho
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,BOGOTÁ. D.C.,SOGAMOSO,8,TRANSITO,2,0,12:00:00,AM,03,01,2021
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,BOGOTÁ. D.C.,CALI,11,ORIGEN,4,45,12:00:00,AM,03,12,2021
2,T.T. DE CALI,BUSETA,BASICO,CALI,LA CRUZ,6,ORIGEN,1,3,12:00:00,AM,01,09,2021
3,T.T. DE MELGAR,BUS,BASICO,IBAGUÉ,BOGOTÁ. D.C.,16,TRANSITO,1,10,12:00:00,AM,08,13,2021
4,T.T. DE MELGAR,MICROBUS,BASICO,MELGAR,NILO,12,ORIGEN,1,1,12:00:00,AM,02,28,2021


* Convertir las nuevas columnas a enteros

In [15]:
datos['dia_despacho'] = datos['dia_despacho'].astype("int")
datos['mes_despacho']= datos['mes_despacho'].astype("int")
datos['ano_despacho']= datos['ano_despacho'].astype("int")

*Nota:* Dado que todos los buses salen a la misma hora se eliminara las columnas `hora` y `tipo_hora`

In [16]:
datos.drop(columns=['hora','tipo_hora'],inplace=True)
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,dia_despacho,mes_despacho,ano_despacho
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,BOGOTÁ. D.C.,SOGAMOSO,8,TRANSITO,2,0,3,1,2021
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,BOGOTÁ. D.C.,CALI,11,ORIGEN,4,45,3,12,2021
2,T.T. DE CALI,BUSETA,BASICO,CALI,LA CRUZ,6,ORIGEN,1,3,1,9,2021
3,T.T. DE MELGAR,BUS,BASICO,IBAGUÉ,BOGOTÁ. D.C.,16,TRANSITO,1,10,8,13,2021
4,T.T. DE MELGAR,MICROBUS,BASICO,MELGAR,NILO,12,ORIGEN,1,1,2,28,2021


**4. Encoding de datos *Categoricos***
****


* **Deteccion de datos categoricos**: Estos se identificaran bajo el nombre de *Object*

In [17]:
tipologia_datos=dict(datos.dtypes)
datos.dtypes

TERMINAL                  object
CLASE_VEHICULO            object
NIVEL_SERVICIO            object
MUNICIPIO_ORIGEN_RUTA     object
MUNICIPIO_DESTINO_RUTA    object
HORA_DESPACHO              int64
TIPO_DESPACHO             object
DESPACHOS                  int64
PASAJEROS                  int64
dia_despacho               int32
mes_despacho               int32
ano_despacho               int32
dtype: object

* **Encoding de datos Categoricos y creaccion de susrespectivos diccionarios**

In [18]:
def Transform_Nominal(datos_columna,nombre_col):
  datos_col_unicos =datos_columna.unique()
  dicc_generico = {v: k for k, v in dict(enumerate(datos_col_unicos)).items()}

  #Cambio de categorico a nominal
  datos[nombre_col] = [dicc_generico[value] for value in datos_columna]

  return dicc_generico




def Categorical_Nominal(tipologia:dict):
  keys = tipologia.keys()
  diccionario_global = {}

  #Buscar datos categoricos
  for key in keys:
    if tipologia[key] == 'O':
      #Almacenar variables Categoricas en la lista
      list_dicc_col = Transform_Nominal(datos[key],key)
      diccionario_global.setdefault(key,list_dicc_col)
  return diccionario_global


In [19]:
dicc_json=Categorical_Nominal(tipologia_datos)

In [20]:
datos.dtypes

TERMINAL                  int64
CLASE_VEHICULO            int64
NIVEL_SERVICIO            int64
MUNICIPIO_ORIGEN_RUTA     int64
MUNICIPIO_DESTINO_RUTA    int64
HORA_DESPACHO             int64
TIPO_DESPACHO             int64
DESPACHOS                 int64
PASAJEROS                 int64
dia_despacho              int32
mes_despacho              int32
ano_despacho              int32
dtype: object

* Demostracion del estado de los datos

In [21]:
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,dia_despacho,mes_despacho,ano_despacho
0,0,0,0,0,0,8,0,2,0,3,1,2021
1,1,1,0,0,1,11,1,4,45,3,12,2021
2,2,2,1,1,2,6,1,1,3,1,9,2021
3,3,1,1,2,3,16,0,1,10,8,13,2021
4,3,0,1,3,4,12,1,1,1,2,28,2021


## **Seeccion 3** Creacion del modelo y distribucion de datos
*****
Para esta seccion dividiremos los datos y crearemos un modelo que con base al *municipio origne*,*municipio destino*, *Tipo vehiculo* y *Clase del vehiculo* 
prediga *numero pasajeros* y *Duracion del viaje*

In [22]:
from sklearn.model_selection import train_test_split
import keras as kr
import json


* Datos a predecir


In [25]:
datos_y = datos[['PASAJEROS','HORA_DESPACHO']]
datos.drop(columns=['HORA_DESPACHO','PASAJEROS'],inplace=True)

* Datos de entrada

In [26]:
datos_x = datos[['MUNICIPIO_ORIGEN_RUTA','MUNICIPIO_DESTINO_RUTA','CLASE_VEHICULO','NIVEL_SERVICIO']]
datos.drop(columns=['MUNICIPIO_ORIGEN_RUTA','MUNICIPIO_DESTINO_RUTA','CLASE_VEHICULO','NIVEL_SERVICIO'],inplace=True)

* Funcion para dividir los datos

In [27]:
x_train,x_test,y_train,y_test =train_test_split(np.array(datos_x),np.array(datos_y))

### **2. Creacion del modelo**
****

In [30]:
modelo = kr.Sequential([kr.layers.Dense(16,input_shape=(4,)),
         kr.layers.Dense(10,activation='relu'),
         kr.layers.Dense(20,activation='relu'),
         kr.layers.Dense(2)
        ]) 
modelo.summary()
modelo.compile(loss='mse',optimizer='adam',metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                80        
                                                                 
 dense_5 (Dense)             (None, 10)                170       
                                                                 
 dense_6 (Dense)             (None, 20)                220       
                                                                 
 dense_7 (Dense)             (None, 2)                 42        
                                                                 
Total params: 512
Trainable params: 512
Non-trainable params: 0
_________________________________________________________________


In [32]:
modelo.fit(x_train,y_train,epochs=10,batch_size=642)

Epoch 1/10
12950/12950 [==============================] - 16s 1ms/step - loss: 218.4301 - acc: 0.5835
Epoch 2/10
12950/12950 [==============================] - 13s 991us/step - loss: 218.1586 - acc: 0.5838
Epoch 3/10
12950/12950 [==============================] - 15s 1ms/step - loss: 217.6142 - acc: 0.5849
Epoch 4/10
12950/12950 [==============================] - 18s 1ms/step - loss: 217.0321 - acc: 0.5872
Epoch 5/10
12950/12950 [==============================] - 17s 1ms/step - loss: 216.7445 - acc: 0.5889
Epoch 6/10
12950/12950 [==============================] - 14s 1ms/step - loss: 216.4682 - acc: 0.5908
Epoch 7/10
12950/12950 [==============================] - 16s 1ms/step - loss: 216.1349 - acc: 0.5935
Epoch 8/10
12950/12950 [==============================] - 16s 1ms/step - loss: 216.0315 - acc: 0.5933
Epoch 9/10
12950/12950 [==============================] - 16s 1ms/step - loss: 215.8636 - acc: 0.5933
Epoch 10/10
12950/12950 [==============================] - 15s 1ms/step - loss: 

In [49]:
modelo.evaluate(x_test,y_test)

86602/86602 [==============================] - 103s 1ms/step - loss: 240.2151 - acc: 0.5402


[240.21510314941406, 0.5402258634567261]

**Guardar diccionario de los datos**
****

In [23]:
json_dict = json.dumps(dicc_json)
with open('habilidad_extra/diccionario.json','wb') as file:
        file.write(json_dict.encode('utf-8'))

**Guardar modelo**
****

In [50]:
modelo.save("modelo_Pinbus.h5")

* Preparar el modelo para js

In [51]:
! tensorflowjs_converter --input_format keras modelo_Pinbus.h5 habilidad_extra/modelo_js
! DEL modelo_Pinbus.h5
